In [2]:
# Pytorch utilities
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader

from scipy.spatial.transform import Rotation as R
#from sklearn.linear_model import LinearRegression
#from sklearn.preprocessing import PolynomialFeatures
#from sklearn.pipeline import make_pipeline
import imageio

# Plotting utilities
%matplotlib widget
import matplotlib
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from IPython.display import HTML
import matplotlib.animation as animation
from torch.utils.tensorboard import SummaryWriter
from timeit import default_timer as timer
import pyprind

# Directory and file utilities
from os import listdir
from os.path import isfile, isdir, join
import json

In [2]:
with open('../../results/small_body_S.json', 'r') as j:
    jd = json.load(j)
    tr, val, test = jd['train'], jd['validation'], jd['test']
    tr_inputs, tr_predictions, tr_groundtruth, tr_lengths, tr_inp_scale, tr_out_scale, tr_mx, tr_my, tr_mz = tuple(torch.tensor(tr[n]) for n in ['inputs', 'predictions',
                                                                                              'labels', 'lengths',
                                                                                               'is', 'os', 'mx', 'my', 'mz'])
    val_inputs, val_predictions, val_groundtruth, val_length, val_inp_scale, val_out_scale, val_mx, val_my, val_mz = tuple(torch.tensor(val[n]) for n in ['inputs', 'predictions',
                                                                                                  'labels', 'lengths',
                                                                                                  'is', 'os', 'mx', 'my', 'mz'])
    test_inputs, test_predictions, test_groundtruth, test_lengths, test_inp_scale, test_out_scale, test_mx, test_my, test_mz = tuple(torch.tensor(test[n]) for n in ['inputs', 'predictions',
                                                                                                         'labels', 'lengths',
                                                                                                         'is', 'os', 'mx', 'my', 'mz'])

In [10]:
tr_inputs.shape, val_predictions.shape, test_lengths.shape

(torch.Size([288, 250, 52]), torch.Size([32, 250, 26]), torch.Size([32]))

In [16]:
frames = [i for i in range(1,9,2)]
video_n = 29

c_inputs = test_inputs.clone()
c_output = test_predictions.clone()
c_labels = test_groundtruth.clone()


In [17]:
for vid in range(c_labels.shape[0]): 
    c_inputs[vid,:,::2].mul_(test_inp_scale[vid, 0])
    c_inputs[vid,:,1::2].mul_(test_inp_scale[vid, 1])
    c_output[vid].mul_(test_out_scale[vid,0])
    c_labels[vid].mul_(test_out_scale[vid,1])
    
    c_inputs[vid,:,::2].mul_(test_mx[vid, 1])
    c_inputs[vid,:,1::2].mul_(test_my[vid, 1])
    c_output[vid].mul_(test_mz[vid,0])
    c_labels[vid].mul_(test_mz[vid,1])

In [18]:
from matplotlib.animation import FuncAnimation

frames = [i for i in range(1,150)]
inp = c_inputs
preds = c_labels
bodiesXY = torch.chunk(inp[video_n, frames, :], len(frames), dim=0)
pred_bodiesZ = torch.chunk(preds[video_n, frames, :], len(frames), dim=0)
limset = False
global lims

In [19]:
plt.close("all")

In [ ]:
def update(j):
    fig = plt.figure()
    fig.set_tight_layout(True)
    ax = fig.add_subplot(111, projection='3d')
    label = 'timestep {0}'.format(j)
    # Update the line and the axes (with a new xlabel). Return a tuple of
    # "artists" that have to be redrawn for this frame.
    
    x = bodiesXY[j].squeeze()[::2].tolist()
    y = bodiesXY[j].squeeze()[1::2].tolist()
    pred_z = pred_bodiesZ[j].squeeze().tolist()

    r = R.from_euler('y', -60, degrees=True)

    xyz1 = np.asarray([c for c in zip(x, y, pred_z)])
    xyz1 = r.apply(xyz1)
    x1 = xyz1[:,0]
    y1 = xyz1[:,1]
    pred_z = xyz1[:,2]

    r_arm = [[c[i] for i in [1, 0, 9, 10, 11]] for c in [x1, y1, pred_z]]
    l_arm = [[c[i] for i in [0, 3, 4, 5]] for c in [x1, y1, pred_z]]
    r_leg = [[c[i] for i in [0, 2, 12, 13, 14, 22, 23, 24]] for c in [x1, y1, pred_z]]
    l_leg = [[c[i] for i in [2, 6, 7, 8, 19, 20, 21]] for c in [x1, y1, pred_z]]
    head = [[c[i] for i in [18, 17, 1, 15, 16]] for c in [x1, y1, pred_z]]

    ax.plot(r_arm[0], r_arm[1], r_arm[2])
    ax.plot(l_arm[0], l_arm[1], l_arm[2])
    ax.plot(r_leg[0], r_leg[1], r_leg[2])
    ax.plot(l_leg[0], l_leg[1], l_leg[2])
    ax.plot(head[0], head[1], head[2])
    
    global limset
    if not limset:
        global lims
        lims = ax.get_xlim(), ax.get_ylim(), ax.get_zlim()
        limset = True
    spans = lims[0][1]-lims[0][0], lims[1][1]-lims[1][0], lims[2][1]-lims[2][0]
    span = max(spans)
    margins = [(span-s)/2 for  s in spans]
    ax.set_xlim(lims[0][0]-margins[0], lims[0][1]+margins[0])
    ax.set_ylim(lims[1][0]-margins[1], lims[1][1]+margins[1])
    ax.set_zlim(lims[2][0]-margins[2], lims[2][1]+margins[2])

    ax.view_init(elev=-65., azim=-90.)
    
    ax.set_xlabel(label)
    
    # Used to return the plot as an image rray
    fig.canvas.draw()       # draw the canvas, cache the renderer
    image = np.frombuffer(fig.canvas.tostring_rgb(), dtype='uint8')
    image  = image.reshape(fig.canvas.get_width_height()[::-1] + (3,))

    return image

kwargs_write = {'fps':10.0, 'quantizer':'nq'}
imageio.mimsave('./test_bodyS_true.gif', [update(i) for i in range(148)], fps=15)

In [46]:
with open('../../results/small_face_S.json', 'r') as j:
    jd = json.load(j)
    tr, val, test = jd['train'], jd['validation'], jd['test']
    tr_inputs, tr_predictions, tr_groundtruth, tr_lengths, tr_inp_scale, tr_out_scale, tr_mx, tr_my, tr_mz = tuple(torch.tensor(tr[n]) for n in ['inputs', 'predictions',
                                                                                              'labels', 'lengths',
                                                                                               'is', 'os', 'mx', 'my', 'mz'])
    val_inputs, val_predictions, val_groundtruth, val_length, val_inp_scale, val_out_scale, val_mx, val_my, val_mz = tuple(torch.tensor(val[n]) for n in ['inputs', 'predictions',
                                                                                                  'labels', 'lengths',
                                                                                                  'is', 'os', 'mx', 'my', 'mz'])
    test_inputs, test_predictions, test_groundtruth, test_lengths, test_inp_scale, test_out_scale, test_mx, test_my, test_mz = tuple(torch.tensor(test[n]) for n in ['inputs', 'predictions',
                                                                                                         'labels', 'lengths',
                                                                                                         'is', 'os', 'mx', 'my', 'mz'])

In [47]:
tr_inputs.shape, val_predictions.shape, test_lengths.shape

(torch.Size([288, 250, 140]), torch.Size([32, 250, 70]), torch.Size([32]))

In [48]:
frames = [i for i in range(1,9,2)]
video_n = 23

c_inputs = test_inputs.clone()
c_output = test_predictions.clone()
c_labels = test_groundtruth.clone()


In [49]:
for vid in range(c_labels.shape[0]): 
    c_inputs[vid,:,::2].mul_(test_inp_scale[vid, 0])
    c_inputs[vid,:,1::2].mul_(test_inp_scale[vid, 1])
    c_output[vid].mul_(test_out_scale[vid,0])
    c_labels[vid].mul_(test_out_scale[vid,1])
    
    c_inputs[vid,:,::2].mul_(test_mx[vid, 1])
    c_inputs[vid,:,1::2].mul_(test_my[vid, 1])
    c_output[vid].mul_(test_mz[vid,0])
    c_labels[vid].mul_(test_mz[vid,1])

In [50]:
from matplotlib.animation import FuncAnimation

frames = [i for i in range(1,150)]
inp = c_inputs
preds = c_labels
bodiesXY = torch.chunk(inp[video_n, frames, :], len(frames), dim=0)
pred_bodiesZ = torch.chunk(preds[video_n, frames, :], len(frames), dim=0)
limset = False
global lims

In [51]:
plt.close("all")

In [ ]:
def update(j):
    fig = plt.figure()
    fig.set_tight_layout(True)
    ax = fig.add_subplot(111, projection='3d')
    label = 'timestep {0}'.format(j)
    # Update the line and the axes (with a new xlabel). Return a tuple of
    # "artists" that have to be redrawn for this frame.
    
    x = bodiesXY[j].squeeze()[::2].tolist()
    y = bodiesXY[j].squeeze()[1::2].tolist()
    pred_z = pred_bodiesZ[j].squeeze().tolist()

    r = R.from_euler('y', -60, degrees=True)

    xyz1 = np.asarray([c for c in zip(x, y, pred_z)])
    xyz1 = r.apply(xyz1)
    x1 = xyz1[:,0]
    y1 = xyz1[:,1]
    pred_z = xyz1[:,2]

    r_eyebrow = [[c[i] for i in range(17, 22)] for c in [x1, y1, pred_z]]
    l_eyebrow = [[c[i] for i in range(22, 27)] for c in [x1, y1, pred_z]]
    l_eye = [[c[i] for i in range(42, 48)] for c in [x1, y1, pred_z]]
    r_eye = [[c[i] for i in range(36, 42)] for c in [x1, y1, pred_z]]
    nose1 = [[c[i] for i in range(27, 31)] for c in [x1, y1, pred_z]]
    nose2 = [[c[i] for i in range(31, 36)] for c in [x1, y1, pred_z]]
    ext_mouth = [[c[i] for i in range(48, 60)] for c in [x1, y1, pred_z]]
    int_mouth = [[c[i] for i in range(60, 68)] for c in [x1, y1, pred_z]]
    contour = [[c[i] for i in range(0, 17)] for c in [x1, y1, pred_z]]

    ax.plot(r_eyebrow[0], r_eyebrow[1], r_eyebrow[2])
    ax.plot(l_eyebrow[0], l_eyebrow[1], l_eyebrow[2])
    ax.plot(l_eye[0], l_eye[1], l_eye[2])
    ax.plot(r_eye[0], r_eye[1], r_eye[2])
    ax.plot(nose1[0], nose1[1], nose1[2])
    ax.plot(nose2[0], nose2[1], nose2[2])
    ax.plot(ext_mouth[0], ext_mouth[1], ext_mouth[2])
    ax.plot(int_mouth[0], int_mouth[1], int_mouth[2])
    
    global limset
    if not limset:
        global lims
        lims = ax.get_xlim(), ax.get_ylim(), ax.get_zlim()
        limset = True
    spans = lims[0][1]-lims[0][0], lims[1][1]-lims[1][0], lims[2][1]-lims[2][0]
    span = max(spans)
    margins = [(span-s)/2 for  s in spans]
    ax.set_xlim(lims[0][0]-margins[0], lims[0][1]+margins[0])
    ax.set_ylim(lims[1][0]-margins[1], lims[1][1]+margins[1])
    ax.set_zlim(lims[2][0]-margins[2], lims[2][1]+margins[2])

    ax.view_init(elev=-65., azim=-90.)
    
    ax.set_xlabel(label)
    
    # Used to return the plot as an image rray
    fig.canvas.draw()       # draw the canvas, cache the renderer
    image = np.frombuffer(fig.canvas.tostring_rgb(), dtype='uint8')
    image  = image.reshape(fig.canvas.get_width_height()[::-1] + (3,))

    return image

kwargs_write = {'fps':2.0, 'quantizer':'nq'}
imageio.mimsave('./test_faceS_true.gif', [update(i) for i in range(148)], fps=15)

In [20]:
with open('../../results/small_hands_S.json', 'r') as j:
    jd = json.load(j)
    tr, val, test = jd['train'], jd['validation'], jd['test']
    tr_inputs, tr_predictions, tr_groundtruth, tr_lengths, tr_inp_scale, tr_out_scale, tr_mx, tr_my, tr_mz = tuple(torch.tensor(tr[n]) for n in ['inputs', 'predictions',
                                                                                              'labels', 'lengths',
                                                                                               'is', 'os', 'mx', 'my', 'mz'])
    val_inputs, val_predictions, val_groundtruth, val_length, val_inp_scale, val_out_scale, val_mx, val_my, val_mz = tuple(torch.tensor(val[n]) for n in ['inputs', 'predictions',
                                                                                                  'labels', 'lengths',
                                                                                                  'is', 'os', 'mx', 'my', 'mz'])
    test_inputs, test_predictions, test_groundtruth, test_lengths, test_inp_scale, test_out_scale, test_mx, test_my, test_mz = tuple(torch.tensor(test[n]) for n in ['inputs', 'predictions',
                                                                                                         'labels', 'lengths',
                                                                                                         'is', 'os', 'mx', 'my', 'mz'])

In [21]:
tr_inputs.shape, val_predictions.shape, test_lengths.shape

(torch.Size([300, 250, 84]), torch.Size([25, 250, 42]), torch.Size([25]))

In [22]:
frames = [i for i in range(1,9,2)]
video_n = 17

c_inputs = test_inputs.clone()
c_output = test_predictions.clone()
c_labels = test_groundtruth.clone()


In [23]:
for vid in range(c_labels.shape[0]): 
    c_inputs[vid,:,::2].mul_(test_inp_scale[vid, 0])
    c_inputs[vid,:,1::2].mul_(test_inp_scale[vid, 1])
    c_output[vid].mul_(test_out_scale[vid,0])
    c_labels[vid].mul_(test_out_scale[vid,1])
    
    c_inputs[vid,:,::2].mul_(test_mx[vid, 1])
    c_inputs[vid,:,1::2].mul_(test_my[vid, 1])
    c_output[vid].mul_(test_mz[vid,0])
    c_labels[vid].mul_(test_mz[vid,1])

In [27]:
from matplotlib.animation import FuncAnimation

frames = [i for i in range(1,150)]
inp = c_inputs
preds = c_labels
bodiesXY = torch.chunk(inp[video_n, frames, :], len(frames), dim=0)
pred_bodiesZ = torch.chunk(preds[video_n, frames, :], len(frames), dim=0)
limset = False
global lims

In [28]:
plt.close("all")

In [ ]:
def update(j):
    fig = plt.figure()
    fig.set_tight_layout(True)
    ax = fig.add_subplot(111, projection='3d')
    label = 'timestep {0}'.format(j)
    # Update the line and the axes (with a new xlabel). Return a tuple of
    # "artists" that have to be redrawn for this frame.
    
    x = bodiesXY[j].squeeze()[::2].tolist()
    y = bodiesXY[j].squeeze()[1::2].tolist()
    pred_z = pred_bodiesZ[j].squeeze().tolist()

    r = R.from_euler('y', -60, degrees=True)

    xyz1 = np.asarray([c for c in zip(x, y, pred_z)])
    xyz1 = r.apply(xyz1)
    x1 = xyz1[:,0]
    y1 = xyz1[:,1]
    pred_z = xyz1[:,2]

    rh0 = [[c[i] for i in [0, 1, 2, 3, 4]] for c in [x1, y1, pred_z]]
    rh1 = [[c[i] for i in [0, 5, 6, 7, 8]] for c in [x1, y1, pred_z]]
    rh2 = [[c[i] for i in [0, 9, 10, 11, 12]] for c in [x1, y1, pred_z]]
    rh3 = [[c[i] for i in [0, 13, 14, 15, 16]] for c in [x1, y1, pred_z]]
    rh4 = [[c[i] for i in [0, 17, 18, 19, 20]] for c in [x1, y1, pred_z]]
    lh0 = [[c[i+21] for i in [0, 1, 2, 3, 4]] for c in [x1, y1, pred_z]]
    lh1 = [[c[i+21] for i in [0, 5, 6, 7, 8]] for c in [x1, y1, pred_z]]
    lh2 = [[c[i+21] for i in [0, 9, 10, 11, 12]] for c in [x1, y1, pred_z]]
    lh3 = [[c[i+21] for i in [0, 13, 14, 15, 16]] for c in [x1, y1, pred_z]]
    lh4 = [[c[i+21] for i in [0, 17, 18, 19, 20]] for c in [x1, y1, pred_z]]

    ax.plot(rh0[0], rh0[1], rh0[2])
    ax.plot(rh1[0], rh1[1], rh1[2])
    ax.plot(rh2[0], rh2[1], rh2[2])
    ax.plot(rh3[0], rh3[1], rh3[2])
    ax.plot(rh4[0], rh4[1], rh4[2])

    ax.plot(lh0[0], lh0[1], lh0[2])
    ax.plot(lh1[0], lh1[1], lh1[2])
    ax.plot(lh2[0], lh2[1], lh2[2])
    ax.plot(lh3[0], lh3[1], lh3[2])
    ax.plot(lh4[0], lh4[1], lh4[2])
    
    global limset
    if not limset:
        global lims
        lims = ax.get_xlim(), ax.get_ylim(), ax.get_zlim()
        limset = True
    spans = lims[0][1]-lims[0][0], lims[1][1]-lims[1][0], lims[2][1]-lims[2][0]
    span = max(spans)
    margins = [(span-s)/2 for  s in spans]
    ax.set_xlim(lims[0][0]-margins[0], lims[0][1]+margins[0])
    ax.set_ylim(lims[1][0]-margins[1], lims[1][1]+margins[1])
    ax.set_zlim(lims[2][0]-margins[2], lims[2][1]+margins[2])

    ax.view_init(elev=-65., azim=-90.)
    
    ax.set_xlabel(label)
    
    # Used to return the plot as an image rray
    fig.canvas.draw()       # draw the canvas, cache the renderer
    image = np.frombuffer(fig.canvas.tostring_rgb(), dtype='uint8')
    image  = image.reshape(fig.canvas.get_width_height()[::-1] + (3,))

    return image

kwargs_write = {'fps':2.0, 'quantizer':'nq'}
imageio.mimsave('./test_handsS_true.gif', [update(i) for i in range(148)], fps=15)

In [52]:
with open('../../results/small_all_S.json', 'r') as j:
    jd = json.load(j)
    tr, val, test = jd['train'], jd['validation'], jd['test']
    tr_inputs, tr_predictions, tr_groundtruth, tr_lengths, tr_inp_scale, tr_out_scale, tr_mx, tr_my, tr_mz = tuple(torch.tensor(tr[n]) for n in ['inputs', 'predictions',
                                                                                              'labels', 'lengths',
                                                                                               'is', 'os', 'mx', 'my', 'mz'])
    val_inputs, val_predictions, val_groundtruth, val_length, val_inp_scale, val_out_scale, val_mx, val_my, val_mz = tuple(torch.tensor(val[n]) for n in ['inputs', 'predictions',
                                                                                                  'labels', 'lengths',
                                                                                                  'is', 'os', 'mx', 'my', 'mz'])
    test_inputs, test_predictions, test_groundtruth, test_lengths, test_inp_scale, test_out_scale, test_mx, test_my, test_mz = tuple(torch.tensor(test[n]) for n in ['inputs', 'predictions',
                                                                                                         'labels', 'lengths',
                                                                                                         'is', 'os', 'mx', 'my', 'mz'])

In [53]:
tr_inputs.shape, val_predictions.shape, test_lengths.shape

(torch.Size([288, 250, 276]), torch.Size([32, 250, 138]), torch.Size([32]))

In [54]:
video_n = 23

c_inputs = test_inputs.clone()
c_output = test_predictions.clone()
c_labels = test_groundtruth.clone()


In [55]:
for vid in range(c_labels.shape[0]): 
    c_inputs[vid,:,::2].mul_(test_inp_scale[vid, 0])
    c_inputs[vid,:,1::2].mul_(test_inp_scale[vid, 1])
    c_output[vid].mul_(test_out_scale[vid,0])
    c_labels[vid].mul_(test_out_scale[vid,1])
    
    c_inputs[vid,:,::2].mul_(test_mx[vid, 1])
    c_inputs[:,:,1::2].mul_(test_my[vid, 1])
    c_output[vid].mul_(test_mz[vid,0]*10)
    c_labels[vid].mul_(test_mz[vid,1])

In [59]:
from matplotlib.animation import FuncAnimation

frames = [i for i in range(150)]
inp = c_inputs
preds = c_labels
bodiesXY = torch.chunk(inp[video_n, frames, :], len(frames), dim=0)
pred_bodiesZ = torch.chunk(preds[video_n, frames, :], len(frames), dim=0)
limset = False
global lims

In [60]:
plt.close("all")

In [61]:
def update(j):
    fig = plt.figure()
    ax = fig.add_subplot(111, projection='3d')
    label = 'timestep {0}'.format(j)
    # Update the line and the axes (with a new xlabel). Return a tuple of
    # "artists" that have to be redrawn for this frame.
    
    x = bodiesXY[j].squeeze()[::2].tolist()
    y = bodiesXY[j].squeeze()[1::2].tolist()
    pred_z = pred_bodiesZ[j].squeeze().tolist()

    r = R.from_euler('y', -60, degrees=True)

    xyz1 = np.asarray([c for c in zip(x, y, pred_z)])
    xyz1 = r.apply(xyz1)
    x1 = xyz1[:,0]
    y1 = xyz1[:,1]
    pred_z = xyz1[:,2]

    r_eyebrow = [[c[i] for i in range(17, 22)] for c in [x1, y1, pred_z]]
    l_eyebrow = [[c[i] for i in range(22, 27)] for c in [x1, y1, pred_z]]
    l_eye = [[c[i] for i in range(42, 48)] for c in [x1, y1, pred_z]]
    r_eye = [[c[i] for i in range(36, 42)] for c in [x1, y1, pred_z]]
    nose1 = [[c[i] for i in range(27, 31)] for c in [x1, y1, pred_z]]
    nose2 = [[c[i] for i in range(31, 36)] for c in [x1, y1, pred_z]]
    ext_mouth = [[c[i] for i in range(48, 60)] for c in [x1, y1, pred_z]]
    int_mouth = [[c[i] for i in range(60, 68)] for c in [x1, y1, pred_z]]
    contour = [[c[i] for i in range(0, 17)] for c in [x1, y1, pred_z]]
    
    rh0 = [[c[i+70] for i in [0, 1, 2, 3, 4]] for c in [x1, y1, pred_z]]
    rh1 = [[c[i+70] for i in [0, 5, 6, 7, 8]] for c in [x1, y1, pred_z]]
    rh2 = [[c[i+70] for i in [0, 9, 10, 11, 12]] for c in [x1, y1, pred_z]]
    rh3 = [[c[i+70] for i in [0, 13, 14, 15, 16]] for c in [x1, y1, pred_z]]
    rh4 = [[c[i+70] for i in [0, 17, 18, 19, 20]] for c in [x1, y1, pred_z]]
    lh0 = [[c[i+91] for i in [0, 1, 2, 3, 4]] for c in [x1, y1, pred_z]]
    lh1 = [[c[i+91] for i in [0, 5, 6, 7, 8]] for c in [x1, y1, pred_z]]
    lh2 = [[c[i+91] for i in [0, 9, 10, 11, 12]] for c in [x1, y1, pred_z]]
    lh3 = [[c[i+91] for i in [0, 13, 14, 15, 16]] for c in [x1, y1, pred_z]]
    lh4 = [[c[i+91] for i in [0, 17, 18, 19, 20]] for c in [x1, y1, pred_z]]
    

    r_arm = [[c[i+112] for i in [1, 0, 9, 10, 11]] for c in [x1, y1, pred_z]]
    l_arm = [[c[i+112] for i in [0, 3, 4, 5]] for c in [x1, y1, pred_z]]
    r_leg = [[c[i+112] for i in [0, 2, 12, 13, 14, 22, 23, 24]] for c in [x1, y1, pred_z]]
    l_leg = [[c[i+112] for i in [2, 6, 7, 8, 19, 20, 21]] for c in [x1, y1, pred_z]]
    head = [[c[i+112] for i in [18, 17, 1, 15, 16]] for c in [x1, y1, pred_z]]

    ax.plot(r_eyebrow[0], r_eyebrow[1], r_eyebrow[2])
    ax.plot(l_eyebrow[0], l_eyebrow[1], l_eyebrow[2])
    ax.plot(l_eye[0], l_eye[1], l_eye[2])
    ax.plot(r_eye[0], r_eye[1], r_eye[2])
    ax.plot(nose1[0], nose1[1], nose1[2])
    ax.plot(nose2[0], nose2[1], nose2[2])
    ax.plot(ext_mouth[0], ext_mouth[1], ext_mouth[2])
    ax.plot(int_mouth[0], int_mouth[1], int_mouth[2])
    
    ax.plot(rh0[0], rh0[1], rh0[2])
    ax.plot(rh1[0], rh1[1], rh1[2])
    ax.plot(rh2[0], rh2[1], rh2[2])
    ax.plot(rh3[0], rh3[1], rh3[2])
    ax.plot(rh4[0], rh4[1], rh4[2])

    ax.plot(lh0[0], lh0[1], lh0[2])
    ax.plot(lh1[0], lh1[1], lh1[2])
    ax.plot(lh2[0], lh2[1], lh2[2])
    ax.plot(lh3[0], lh3[1], lh3[2])
    ax.plot(lh4[0], lh4[1], lh4[2])
    
    ax.plot(r_arm[0], r_arm[1], r_arm[2])
    ax.plot(l_arm[0], l_arm[1], l_arm[2])
    ax.plot(r_leg[0], r_leg[1], r_leg[2])
    ax.plot(l_leg[0], l_leg[1], l_leg[2])
    ax.plot(head[0], head[1], head[2])
    
    global limset
    if not limset:
        global lims
        lims = ax.get_xlim(), ax.get_ylim(), ax.get_zlim()
        limset = True
    spans = lims[0][1]-lims[0][0], lims[1][1]-lims[1][0], lims[2][1]-lims[2][0]
    span = max(spans)
    margins = [(span-s)/2 for  s in spans]
    ax.set_xlim(lims[0][0]-margins[0], lims[0][1]+margins[0])
    ax.set_ylim(lims[1][0]-margins[1], lims[1][1]+margins[1])
    ax.set_zlim(lims[2][0]-margins[2], lims[2][1]+margins[2])

    ax.view_init(elev=-65., azim=-90.)
    
    ax.set_xlabel(label)
    
    # Used to return the plot as an image rray
    fig.canvas.draw()       # draw the canvas, cache the renderer
    image = np.frombuffer(fig.canvas.tostring_rgb(), dtype='uint8')
    image  = image.reshape(fig.canvas.get_width_height()[::-1] + (3,))

    return image

kwargs_write = {'fps':2.0, 'quantizer':'nq'}
imageio.mimsave('./test_allS_true.gif', [update(i) for i in range(148)], fps=15)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

...

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:2: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:2: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …